In [1]:
import torch
import torch.nn as nn
import numpy as np
from torchvision import datasets, transforms
import time
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd
import os

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data transformations (same as in training)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),  # Normalize to [-1, 1]
    transforms.Lambda(lambda x: x.view(-1))  # Flatten the image to a vector
])

# Load the test dataset
test_dataset = datasets.KMNIST(root='../data', train=False, transform=transform, download=True)

# DataLoader for test dataset with batch_size=1 to measure time per image
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

# Load the saved classifier model
checkpoint = torch.load('SAE_classifier_best_model.pth', map_location=device)

# Get model configuration
config = checkpoint['config']
encoder_output_size = config['encoder_output_size']
classifier_hidden_sizes = config['classifier_hidden_sizes']
leaky_relu_negative_slope = config['leaky_relu_negative_slope']
batch_norm = config['batch_norm']

# Rebuild the autoencoder encoder
# Load the autoencoder model
autoencoder_checkpoint = torch.load('SAE_best_model.pth', map_location=device)

# Get the autoencoder config
autoencoder_config = autoencoder_checkpoint['config']
input_size = autoencoder_config['input_size']
layer_sizes = autoencoder_config['layer_sizes']
dropout_rates = autoencoder_config['dropout_rates']
activation_functions = [nn.LeakyReLU(negative_slope=0.01) for _ in layer_sizes]  # Assuming LeakyReLU

# Rebuild the autoencoder (from your SAE_model.py)
from SAE_model import StackedAutoencoder

autoencoder = StackedAutoencoder(
    input_size=input_size,
    layer_sizes=layer_sizes,
    activation_functions=activation_functions,
    dropout_rates=dropout_rates,
    weight_init=None  # We don't need to initialize weights, as we'll load them
).to(device)

# Load the autoencoder state dict
autoencoder.load_state_dict(autoencoder_checkpoint['state_dict'])

# Build the classifier model
class SAEClassifier(nn.Module):
    def __init__(self, encoder, encoder_output_size, classifier_hidden_sizes, num_classes, leaky_relu_negative_slope=0.01, batch_norm=True):
        super(SAEClassifier, self).__init__()
        # Encoder (pre-trained)
        self.encoder = encoder  # We will freeze this
        for param in self.encoder.parameters():
            param.requires_grad = False
        # Classifier head
        layers = []
        prev_size = encoder_output_size  # The output size of the encoder
        for idx, hidden_size in enumerate(classifier_hidden_sizes):
            layers.append(nn.Linear(prev_size, hidden_size))
            if batch_norm:
                layers.append(nn.BatchNorm1d(hidden_size))
            layers.append(nn.LeakyReLU(negative_slope=leaky_relu_negative_slope))
            prev_size = hidden_size
        # Output layer
        layers.append(nn.Linear(prev_size, 10))  # KMNIST has 10 classes
        self.classifier = nn.Sequential(*layers)
    def forward(self, x):
        x = self.encoder(x)
        x = self.classifier(x)
        return x

# Instantiate the model
model = SAEClassifier(
    encoder=autoencoder.encoder,
    encoder_output_size=encoder_output_size,
    classifier_hidden_sizes=classifier_hidden_sizes,
    num_classes=10,  # KMNIST has 10 classes
    leaky_relu_negative_slope=leaky_relu_negative_slope,
    batch_norm=batch_norm
).to(device)

# Load the classifier state dict
model.load_state_dict(checkpoint['state_dict'])

# Set the model to evaluation mode
model.eval()

# Evaluate the model on the test data, recording computation time per image
all_labels = []
all_preds = []
computation_times = []

with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Synchronize CUDA for accurate timing if using GPU
        if device.type == 'cuda':
            torch.cuda.synchronize()
        start_time = time.time()
        outputs = model(inputs)
        if device.type == 'cuda':
            torch.cuda.synchronize()
        end_time = time.time()
        computation_time = end_time - start_time
        computation_times.append(computation_time)
        
        _, predicted = torch.max(outputs.data, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Compute confusion matrix
cm = confusion_matrix(all_labels, all_preds)

# Save confusion matrix as CSV
cm_df = pd.DataFrame(cm)
cm_csv_filename = 'confusion_matrix.csv'
cm_df.to_csv(cm_csv_filename, index=False)
print(f"Confusion matrix saved as CSV to {cm_csv_filename}")

# Save confusion matrix as PNG
plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.ylabel('True Labels')
plt.xlabel('Predicted Labels')
plt.title('Confusion Matrix')
cm_png_filename = 'confusion_matrix.png'
plt.savefig(cm_png_filename)
plt.close()
print(f"Confusion matrix saved as PNG to {cm_png_filename}")

# Compute mean and variance of computation time per image
computation_times = np.array(computation_times)
mean_time = np.mean(computation_times)
variance_time = np.var(computation_times)

# Compute accuracy
correct = np.sum(np.array(all_preds) == np.array(all_labels))
total = len(all_labels)
accuracy = 100 * correct / total

# Save computation time statistics and accuracy in a CSV
stats = {
    'mean_time_per_image': [mean_time],
    'variance_time_per_image': [variance_time],
    'accuracy': [accuracy]
}
stats_df = pd.DataFrame(stats)
stats_csv_filename = 'test_evaluation_stats.csv'
stats_df.to_csv(stats_csv_filename, index=False)
print(f"Test evaluation statistics saved to {stats_csv_filename}")
print(f"Accuracy on test set: {accuracy:.2f}%")
print(f"Mean inference time per image: {mean_time:.6f} seconds")
print(f"Variance of inference time per image: {variance_time:.6f} seconds^2")


/scratch/local/50745734/ipykernel_1996409/2446135461.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('SAE_classifier_best_model.pth', map_locati

Confusion matrix saved as CSV to confusion_matrix.csv
Confusion matrix saved as PNG to confusion_matrix.png
Test evaluation statistics saved to test_evaluation_stats.csv
Accuracy on test set: 72.18%
Mean inference time per image: 0.001180 seconds
Variance of inference time per image: 0.000013 seconds^2
